In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
tags = []
types = []

for i in range(1, 447):
    tag_url = f"https://www.onixs.biz/fix-dictionary/4.2/tagNum_{i}.html"
    response = requests.get(tag_url)
    # print(response)
    
    soup = BeautifulSoup(response.text, "html.parser")
    tag_name = soup.find("h1").text.strip().split()[0]
    data_type = soup.find_all("p")[0].text.strip().split()[1]
    tags.append(tag_name)
    types.append(data_type)

In [3]:
d = {"tags": tags, "type":types}
df = pd.DataFrame(d)
df

,tags,type
0,Account,String
1,AdvId,String
2,AdvRefID,String
3,AdvSide,Char
4,AdvTransType,String
...,...,...
441,MultiLegReportingType,char
442,StrikeTime,UTCTimestamp
443,ListStatusText,String
444,EncodedListStatusTextLen,Length


In [5]:
df.value_counts(["type"])

type               
String                 113
int                     88
char                    48
page                    41
Qty                     23
Price                   18
UTCTimestamp            16
Boolean                 15
data                    14
Length                  14
Amt                     13
float                   12
PriceOffset              6
LocalMktDate             5
Exchange                 5
Currency                 4
MultipleValueString      3
MonthYear                2
DayOfMonth               2
Quantity                 1
UTCDate                  1
UTCTimeOnly              1
Char                     1
dtype: int64

Tags by message type (tag 35)

In [89]:
msg_types_1= []
tags_1 = []
fields = []
table_values_1 = []

for i in range(9):
    print(f"message tag {i}")
    tag_url = f"https://www.onixs.biz/fix-dictionary/4.2/msgType_{i}_{i}.html"
    response = requests.get(tag_url)
    # print(response)
    
    soup = BeautifulSoup(response.text, "html.parser")
    msg = soup.find("h1").text.strip().split()[0]
    target_heading = soup.find("h3", text="Structure")
    target_table = target_heading.find_next_sibling("table")
    
    rows = target_table.find_all("tr")
    
    field_values = []
    print(f"row num: {len(rows)}")
    for r in range(2, len(rows)):
        cells = rows[r].find_all("td")
        # print(cells)
        # Process the cell data as needed
        cell_values = []
        for cell in cells:
            cell_values.append(cell.text.strip().split())
        print(f"values length: {len(cell_values)}")
        print(cell_values)
        field_values.append(cell_values)
    table_values_1.append(field_values)
    msg_types_1.append(msg)
    tags_1.append(i)
    
    

message tag 0
row num: 5
values length: 3
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '0']]
values length: 4
[['112'], ['TestReqID'], ['N'], ['Required', 'when', 'the', 'heartbeat', 'is', 'the', 'result', 'of', 'a', 'Test', 'Request', '<1>', 'message.']]
values length: 3
[['<MessageTrailer>'], ['Y'], []]
message tag 1
row num: 5
values length: 3
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '1']]
values length: 4
[['112'], ['TestReqID'], ['Y'], []]
values length: 3
[['<MessageTrailer>'], ['Y'], []]
message tag 2
row num: 6
values length: 3
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '2']]
values length: 4
[['7'], ['BeginSeqNo'], ['Y'], []]
values length: 4
[['16'], ['EndSeqNo'], ['Y'], []]
values length: 3
[['<MessageTrailer>'], ['Y'], []]
message tag 3
row num: 11
values length: 3
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '3']]
values length: 4
[['45'], ['RefSeqNum'], ['Y'], ['MsgSeqNum', '<34>', 'of', 'rejected', 'message']]
values length: 4
[[

In [65]:
# list(range(97, 110))
ascii_list = list(range(97, 110)) + list(range(65, 91))
ascii_list.remove(73) #no 'I', 'U' or 'O' tag
ascii_list.remove(79)
ascii_list.remove(85)
print(ascii_list)

[97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90]


In [90]:
msg_types_2= []
tags_2 = []
table_values_2 = []


for i in ascii_list:
    print(f"message tag {chr(i)}")
    tag_url = f"https://www.onixs.biz/fix-dictionary/4.2/msgType_{chr(i)}_{i}.html"
    response = requests.get(tag_url)
    # print(response)
    
    soup = BeautifulSoup(response.text, "html.parser")
    msg = soup.find("h1").text.strip().split()[0]
    target_heading = soup.find("h3", text="Structure")
    target_table = target_heading.find_next_sibling("table")
    
    rows = target_table.find_all("tr")
    
    field_values = []
    
    print(f"row num: {len(rows)}")
    for r in range(2, len(rows)):
        cells = rows[r].find_all("td")
        # print(cells)
        # Process the cell data as needed
        cell_values = []
        for cell in cells:
            cell_values.append(cell.text.strip().split())
        print(f"values length: {len(cell_values)}")
        print(cell_values)
        field_values.append(cell_values)
    table_values_2.append(field_values)
    msg_types_2.append(msg)
    tags_2.append(chr(i))

message tag a
row num: 26
values length: 3
[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', 'a']]
values length: 4
[['117'], ['QuoteID'], ['N'], []]
values length: 4
[['55'], ['Symbol'], ['Y'], []]
values length: 4
[['65'], ['SymbolSfx'], ['N'], []]
values length: 4
[['48'], ['SecurityID'], ['N'], []]
values length: 4
[['22'], ['IDSource'], ['N'], []]
values length: 4
[['167'], ['SecurityType'], ['N'], ['Must', 'be', 'specified', 'if', 'a', 'Future', 'or', 'Option.', 'If', 'a', 'Future:', 'Symbol', '<55>,', 'SecurityType', '<167>,', 'and', 'MaturityMonthYear', '<200>', 'are', 'required.', 'If', 'an', 'Option:', 'Symbol', '<55>,', 'SecurityType', '<167>,', 'MaturityMonthYear', '<200>,', 'PutOrCall', '<201>,', 'and', 'StrikePrice', '<202>', 'are', 'required.']]
values length: 4
[['200'], ['MaturityMonthYear'], ['N'], ['Specifies', 'the', 'month', 'and', 'year', 'of', 'maturity.', 'Required', 'if', 'MaturityDay', '<205>', 'is', 'specified.']]
values length: 4
[['205'], ['MaturityDay'

In [91]:
msg_types = msg_types_1 + msg_types_2
tag_values = tags_1 + tags_2
table_values = table_values_1 + table_values_2
len(msg_types_1), len(msg_types_2), len(table_values_1), len(table_values_2), len(tags_1), len(tags_2)

(9, 36, 9, 36, 9, 36)

In [92]:
table_values

[[[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '0']],
  [['112'],
   ['TestReqID'],
   ['N'],
   ['Required',
    'when',
    'the',
    'heartbeat',
    'is',
    'the',
    'result',
    'of',
    'a',
    'Test',
    'Request',
    '<1>',
    'message.']],
  [['<MessageTrailer>'], ['Y'], []]],
 [[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '1']],
  [['112'], ['TestReqID'], ['Y'], []],
  [['<MessageTrailer>'], ['Y'], []]],
 [[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '2']],
  [['7'], ['BeginSeqNo'], ['Y'], []],
  [['16'], ['EndSeqNo'], ['Y'], []],
  [['<MessageTrailer>'], ['Y'], []]],
 [[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '3']],
  [['45'],
   ['RefSeqNum'],
   ['Y'],
   ['MsgSeqNum', '<34>', 'of', 'rejected', 'message']],
  [['371'],
   ['RefTagID'],
   ['N'],
   ['The',
    'tag',
    'number',
    'of',
    'the',
    'FIX',
    'field',
    'being',
    'referenced.']],
  [['372'],
   ['RefMsgType'],
   ['N'],
   ['The',
    'MsgType'

In [94]:
print(len(table_values))
for values in table_values:
    print(len(values))
    msg_tags = []
    if len(values) == 3:
        msg_tags.append("header")
    elif len(values) == 4:
        msg_tags.append(values[0][0])
    elif len(values) == 5:
        msg_tags.append(values[1][0])
    else:
        print(f"failed {len(values)}")
        print(values)
    fields.append(msg_tags)

45
3
3
4
9
failed 9
[[['<MessageHeader>'], ['Y'], ['MsgType', '<35>', '=', '3']], [['45'], ['RefSeqNum'], ['Y'], ['MsgSeqNum', '<34>', 'of', 'rejected', 'message']], [['371'], ['RefTagID'], ['N'], ['The', 'tag', 'number', 'of', 'the', 'FIX', 'field', 'being', 'referenced.']], [['372'], ['RefMsgType'], ['N'], ['The', 'MsgType', '<35>', 'of', 'the', 'FIX', 'message', 'being', 'referenced.']], [['373'], ['SessionRejectReason'], ['N'], ['Code', 'to', 'identify', 'reason', 'for', 'a', 'session-level', 'Reject', '<3>', 'message.']], [['58'], ['Text'], ['N'], ['Where', 'possible,', 'message', 'to', 'explain', 'reason', 'for', 'rejection']], [['354'], ['EncodedTextLen'], ['N'], ['Must', 'be', 'set', 'if', 'EncodedText', '<355>', 'field', 'is', 'specified', 'and', 'must', 'immediately', 'precede', 'it.']], [['355'], ['EncodedText'], ['N'], ['Encoded', '(non-ASCII', 'characters)', 'representation', 'of', 'the', 'Text', '<58>', 'field', 'in', 'the', 'encoded', 'format', 'specified', 'via', 'the',